In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from matplotlib import pyplot as plt

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"divyomagarwal","key":"10ab80c2467b1f8a9062e5feecbaa924"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shuvoalok/raf-db-dataset

Dataset URL: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset
License(s): other


In [ ]:
!unzip -q raf-db-dataset.zip -d rafdb_dataset

In [ ]:
import os

In [ ]:
base_dir = '/content/rafdb_dataset'

In [ ]:
dataset_path = '/content/rafdb_dataset/DATASET'
print("Dataset contents:", os.listdir(dataset_path))

Dataset contents: ['train', 'test']


In [ ]:
import pandas as pd

train_labels_df = pd.read_csv(os.path.join(base_dir, 'train_labels.csv'))
print("Train labels sample:\n", train_labels_df.head())

Train labels sample:
                      image  label
0  train_00001_aligned.jpg      5
1  train_00002_aligned.jpg      5
2  train_00003_aligned.jpg      4
3  train_00004_aligned.jpg      4
4  train_00005_aligned.jpg      5


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = os.path.join(dataset_path, 'train')
print("Train directory contents:", os.listdir(train_dir))

Train directory contents: ['5', '1', '3', '4', '7', '2', '6']


In [ ]:
def convert_to_rgb(img):
    if len(img.shape) == 2:  # Grayscale image (H, W)
        img = np.stack([img] * 3, axis=-1)  # Convert to (H, W, 3)
    return img

In [ ]:
val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Match the split defined in train_datagen
    preprocessing_function=convert_to_rgb
)

In [ ]:
train_datagen = ImageDataGenerator(
                rescale=1./255,              # Normalize pixel values to [0, 1]
                validation_split=0.2,        # 20% of training data for validation
                rotation_range=20,           # Randomly rotate images by up to 20 degrees
                width_shift_range=0.2,       # Shift images horizontally by 20% of width
                height_shift_range=0.2,      # Shift images vertically by 20% of height
                horizontal_flip=True,        # Randomly flip images horizontally
                fill_mode='nearest',          # Fill in new pixels after rotation/shift
                preprocessing_function=convert_to_rgb
)




In [ ]:
train_generator = val_test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 9819 images belonging to 7 classes.


In [ ]:
validation_generator = val_test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2452 images belonging to 7 classes.


In [ ]:
test_dir = os.path.join(dataset_path, 'test')

In [ ]:
test_labels_df = pd.read_csv(os.path.join(base_dir, 'test_labels.csv'))
print("Test labels sample:\n", test_labels_df.head())

Test labels sample:
                    image  label
0  test_0001_aligned.jpg      5
1  test_0002_aligned.jpg      1
2  test_0003_aligned.jpg      4
3  test_0004_aligned.jpg      1
4  test_0005_aligned.jpg      5


In [ ]:
test_generator = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Set to False for test data to preserve order for evaluation
)

Found 3068 images belonging to 7 classes.


In [ ]:
print("Train samples:", train_generator.samples)
print("Validation samples:", validation_generator.samples)
print("Test samples:", test_generator.samples)
print("Class indices:", test_generator.class_indices)

Train samples: 9819
Validation samples: 2452
Test samples: 3068
Class indices: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}


In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D

CNN Model 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Number of classes (update if needed, based on your dataset's classes)
num_classes = len(train_generator.class_indices)

# Load EfficientNetB0 base model (without top layers)
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',  # Pretrained on ImageNet
    input_shape=(224, 224, 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularization
predictions = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,058,538 (15.48 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30  # you can increase this depending on performance
)

Epoch 1/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 42s 136ms/step - accuracy: 0.5105 - loss: 1.3453 - val_accuracy: 0.4213 - val_loss: 1.4921
Epoch 2/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 82s 136ms/step - accuracy: 0.5523 - loss: 1.2486 - val_accuracy: 0.5432 - val_loss: 1.2641
Epoch 3/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 42s 137ms/step - accuracy: 0.5725 - loss: 1.1736 - val_accuracy: 0.4976 - val_loss: 1.3381
Epoch 4/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 45s 145ms/step - accuracy: 0.6026 - loss: 1.1208 - val_accuracy: 0.6203 - val_loss: 1.1027
Epoch 5/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 42s 138ms/step - accuracy: 0.6278 - loss: 1.0677 - val_accuracy: 0.6166 - val_loss: 1.0831
Epoch 6/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 43s 141ms/step - accuracy: 0.6492 - loss: 1.0109 - val_accuracy: 0.6060 - val_loss: 1.1173
Epoch 7/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 45s 148ms/step - accuracy: 0.6474 - loss: 0.9776 - val_accuracy: 0.5559 - val_loss: 1.2185
Epoch 8/30
307/307 ━━━━━━━━━━━━━━━━━━━━ 43s 140ms/step - accuracy: 0.6776 - loss: 0

In [1]:
# Unfreeze base model for fine-tuning (optional for better accuracy after initial training)
base_model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5  # add more epochs if needed
)

NameError: name 'base_model' is not defined

In [ ]:
#ACCURACY
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")